# Lab 2

## Experiment Setup

### Import python packages

In [ ]:
# @markdown Run this cell to install dependencies. The whole process will take about 30 seconds.
%%capture

!git clone https://github.com/SafeRoboticsLab/ECE346.git
% cd /content
! git clone https://github.com/mdolab/pyspline.git 
% cd pyspline 
! cp config/defaults/config.LINUX_GFORTRAN.mk config/config.mk 
! make 
! pip install .
% cd /content/ECE346/ROS_ws/src/lab2/traj_planning_ros/src
% pwd

In [ ]:
# @markdown ### load dependency
import os
import time
import numpy as np
import pickle
import csv
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import Image
import imageio
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

from iLQR import Track, iLQR, Dynamics, plot_ellipsoids, Ellipsoid, EllipsoidObj
from iLQR import get_agent_FRS, plot_footprint, state2ell

In [ ]:
#@title { run: "auto" , form-width: "30%"}
#@markdown # Set parameters for the planner

#@markdown ## weights in cost

w_vel =   10# @param {type:"number"}
w_contour = 60 # @param {type:"number"}
w_theta =    4# @param {type:"number"}
w_accel =  0.1  # @param {type:"number"}
w_delta =  0.1  # @param {type:"number"}


#@markdown ## weights in soft constraints

# parameter for soft constraints
q1_v =  0#@param {type:"number"}
q2_v =  0#@param {type:"number"}

q1_road = 2#@param {type:"number"}
q2_road = 5#@param {type:"number"}

q1_obstacle = 3#@param {type:"number"}
q2_obstacle = 5.5#@param {type:"number"}

q1_lat =  1#@param {type:"number"}
q2_lat =  5#@param {type:"number"}


#@markdown ## Uncertainty in simulation
sigma_x = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma_y = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma_v = 0  # @param {type:"slider", min:0, max:0.15, step:0.01}
sigma_theta = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma = np.array([sigma_x, sigma_y, sigma_v, sigma_theta])

#@markdown ## iLQR Parameters
T = 1 #@param {type:"number"}
N = 11 #@param {type:"integer"}
max_itr = 50  #@param {type:"integer"}


params = {
    'wheelbase': 0.257,
    'width': 0.2,
    'length': 0.5,
    'm': 2.99,
    'track_width_R': 0.3,
    'track_width_L': 0.3,
    'track_offset': 0.15,
    'delta_min': -0.35,
    'delta_max': 0.35,
    'v_max': 1,
    'v_min': 0,
    'a_min': -3.5,
    'a_max': 3.5,
    'alat_max': 5,
    'w_vel': w_vel,
    'w_contour': w_contour,
    'w_theta': w_theta,
    'w_accel': w_accel,
    'w_delta': w_delta,
    'q1_v': q1_v,
    'q2_v': q2_v,
    'q1_road': q1_road,
    'q2_road': q2_road,
    'q1_lat': q1_lat,
    'q2_lat': q2_lat,
    'q1_obs': q1_obstacle,
    'q2_obs': q2_obstacle,
    'T': T,
    'N': N,  # the number of planning steps
    'max_itr': max_itr,
    'ego_sigma': sigma,  # the uncertainty in ego agent's dynamics
    'track_file': 'outerloop_center_smooth.csv',
}

In [ ]:
#@title { run: "auto" , form-width: "30%"}
#@markdown # Set experiment tag for snapshot

experiment_tag = "D6"  # @param {type:"string"}

### Load track

In [ ]:
#@title Load Track { display-mode: "form" }
x = []
y = []
# % cd /content/ellReach
with open('iLQR/'+params['track_file'], newline='') as f:
  spamreader = csv.reader(f, delimiter=',')
  for i, row in enumerate(spamreader):
    if i > 0:
      x.append(float(row[0]))
      y.append(float(row[1]))

center_line = np.array([x, y])
track = Track(
    center_line=center_line, width_left=params['track_width_L'],
    width_right=params['track_width_R'], loop=False
)

## Task 1: Static Obstacle

### Initialize

In [ ]:
solver = iLQR(track, params)
pos0, psi0 = track.interp([2])  # The position and yaw on the track.
x_cur = np.array([pos0[0], pos0[1], 0, psi0[-1]])
init_control = np.zeros((2, params['N']))
t_total = 0
plot_cover = False

itr_receding = 150  # The number of receding iterations
state_hist = np.zeros((4, itr_receding))
control_hist = np.zeros((2, itr_receding))
plan_hist = np.zeros((6, params['N'], itr_receding))
K_hist = np.zeros((2, 4, params['N']-1, itr_receding))
fx_hist = np.zeros((4, 4, params['N'], itr_receding))
fu_hist = np.zeros((4, 2, params['N'], itr_receding))

plt.figure(figsize=(15, 15))
#@markdown ## The folder to save figures under /content/figure/
subfolder = "task1_perfect" #@param {type:"string"}
fig_folder = os.path.join("/content", "figure", subfolder)
fig_prog_folder = os.path.join(fig_folder, "progress")
os.makedirs(fig_prog_folder, exist_ok=True)

In [ ]:
# Constructs a static obstacle.
ego_a = params['length'] / 2.0
ego_b = params['width'] / 2.0
ego_q = np.array([0, 5.6])[:, np.newaxis] 
ego_Q = np.diag([ego_a**2, ego_b**2])
static_obs = EllipsoidObj(q=ego_q, Q=ego_Q)
track.plot_track()
static_obs.plot(plt.gca(), color='r', plot_center=False)
static_obs_list = [static_obs for _ in range(params['N'])]

plt.savefig("static_obstacle.png", dpi=200)

### iLQR Planning

In [ ]:
for i in range(itr_receding):
  # Plans the trajectory by using iLQR.
  states, controls, t_process, status, theta, K_closed_loop, fx, fu = (
      solver.solve(x_cur, controls=init_control, obs_list=[static_obs_list])
  )

  # Executes the first control.
  x_cur = solver.dynamics.forward_step(
      x_cur, controls[:, 0], step=10, noise=params['ego_sigma']
  )[0]
  print(
      "[{}]: solver returns status {} and uses {:.3f}.".format(
          i, status, t_process
      ),
      end='\r'
  )
  t_total += t_process

  # Records planning history, states and controls.
  plan_hist[:4, :, i] = states
  plan_hist[4:, :, i] = controls
  state_hist[:, i] = states[:, 0]
  control_hist[:, i] = controls[:, 0]

  K_hist[:, :, :, i] = K_closed_loop
  fx_hist[:, :, :, i] = fx
  fu_hist[:, :, :, i] = fu

  # Updates the nominal control signal for next receding horizon (The first
  # control is executed).
  init_control[:, :-1] = controls[:, 1:]

  # Plots the current progress.
  plt.clf()
  track.plot_track()
  plot_ellipsoids(
      plt.gca(), static_obs_list[0:1], arg_list=[dict(c='r', linewidth=1.)],
      dims=[0, 1], N=50, plot_center=False, use_alpha=True
  )
  plot_ellipsoids(
      plt.gca(), [solver.cost.soft_constraints.ego_ell[0]],
      arg_list=[dict(c='b')], dims=[0, 1], N=50, plot_center=False
  )
  if plot_cover:  # plot circles that cover the footprint.
    static_obs_list[0].plot_circ(plt.gca())
    solver.cost.soft_constraints.ego_ell[0].plot_circ(plt.gca())
  plt.plot(states[0, :], states[1, :], linewidth=2, c='b')
  sc = plt.scatter(
      state_hist[0, :i + 1], state_hist[1, :i + 1], s=24,
      c=state_hist[2, :i + 1], cmap=cm.jet, vmin=0, vmax=params['v_max'],
      edgecolor='none', marker='o'
  )
  cbar = plt.colorbar(sc)
  cbar.set_label(r"velocity [$m/s$]", size=20)
  plt.axis('equal')
  plt.savefig(os.path.join(fig_prog_folder, str(i)+".png"), dpi=200)

plt.close('All')
print("Planning uses {:.3f}.".format(t_total))

### Make experiment snapshot

In [ ]:
import PIL
def save_snapshot(experiment_tag):
  snapshot_folder = os.path.join("/content", "figure", "snapshots")
  os.makedirs(snapshot_folder, exist_ok=True)
  
  img = imageio.imread(os.path.join(fig_prog_folder, str(itr_receding-1) + ".png"))
  imageio.imwrite(os.path.join(snapshot_folder, experiment_tag + ".png"), img[100:700, 220:820])
  imageio.imwrite(os.path.join(snapshot_folder, experiment_tag + "_snap.png"), img[100:200, 400:650])
  
  return img

PIL.Image.fromarray(save_snapshot(experiment_tag)[100:200, 400:650])

In [ ]:
# !zip /content/snapshots.zip /content/figure/snapshots/*

### Visualize

In [ ]:
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
plt.plot(state_hist[0, :], state_hist[1, :], 'b-o', linewidth=3, markersize=8)
plot_footprint(plt.gca(), state_hist[:, :-1], color="b")

plt.title("Trajectory")
plt.savefig(os.path.join(fig_folder, "traj.png"), dpi=200)

# velocity
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
sc = plt.scatter(
    state_hist[0, :], state_hist[1, :], s=80, c=state_hist[2, :], cmap=cm.jet,
    vmin=params['v_min'], vmax=params['v_max'], edgecolor='none', marker='o'
)
cbar = plt.colorbar(sc)
cbar.set_label(r"Velocity [$m/s$]", size=20)
plt.savefig(os.path.join(fig_folder, "vel.png"), dpi=200)

# Longitudinal Accel
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
sc = plt.scatter(
    state_hist[0, :], state_hist[1, :], s=80, c=control_hist[0, :],
    cmap=cm.jet, vmin=params['a_min'], vmax=params['a_max'], edgecolor='none',
    marker='o'
)
cbar = plt.colorbar(sc)
cbar.set_label(r"Longitudinal Accel [$m/s^2$]", size=20)
plt.savefig(os.path.join(fig_folder, "long_acc.png"), dpi=200)

# Longitudinal Accel
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
a_lat = np.abs(state_hist[2, :]**2 * np.tan(control_hist[1, :]) / 0.257)
# print(a_lat)
sc = plt.scatter(
    state_hist[0, :], state_hist[1, :], s=80, c=a_lat, cmap=cm.jet, vmin=0,
    vmax=params['alat_max'], edgecolor='none', marker='o'
)
cbar = plt.colorbar(sc)
cbar.set_label(r"Lateral Accel [$m/s^2$]", size=20)
plt.savefig(os.path.join(fig_folder, "lat_acc.png"), dpi=200)
plt.close("all")

In [ ]:
gif_path = os.path.join(fig_folder, 'rollout.gif')
with imageio.get_writer(gif_path, mode='I') as writer:
  for i in range(itr_receding):
    filename = os.path.join(fig_prog_folder, str(i)+".png")
    image = imageio.imread(filename)
    writer.append_data(image)
Image(open(gif_path,'rb').read(), width=400)

In [ ]:
#@markdown ## Save your trajectory and download it for task2
filepath = os.path.join('/content', 'data')
os.makedirs(filepath, exist_ok=True)
filename = subfolder
filename = os.path.join(filepath, filename+'.p')
pickle.dump(
    [plan_hist, K_hist, fx_hist, fu_hist, params['ego_sigma'], solver.dynamics],
    open(filename, "wb")
)


## Task 1: Simulation Results

### A. Tuning of iLQR parameters
\#| \| |w_vel|w_contour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.891|*initial configuration (bypasses with 0.5)*
A1| \| |4|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |**1.5**|**16**|50| \| |24.998|*bypasses with 0.35, almost gets stuck*

### B. Tuning of weights in cost
\#| \| |w_vel|w_countour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.891|*initial configuration (bypasses with 0.5)*
B1| \| |**10**|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |16.382|*bypasses with 0.7*
B2| \| |**20**|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.922|*bypasses with 0.8*
B3| \| |4|**15**|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |17.713|*bypasses with 0.6, drives more into the other lane*
B4*| \| |4|30|**4**|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |11.153|*brakes shortly, bypasses with 1.0*
B5| \| |4|30|0|**0**|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |16.059|*bypasses with 0.5, stays at slow speed a bit shorter*

### C. Tuning of weights in soft constraint
\#| \| |w_vel|w_countour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.891|*initial configuration (bypasses with 0.5)*
C1| \| |4|30|0|0.1|0.1| \| |0 / 0|**1 / 4**|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |19.007|*bypasses with 0.5, cuts the curves a bit*
C2| \| |4|30|0|0.1|0.1| \| |0 / 0|2 / 5|**1 / 4**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |18.553|*bypasses with 0.7, comes very close to obstacle*

### D. Joint tuning
\#| \| |w_vel|w_countour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.891|*initial configuration (bypasses with 0.5)*
D1| \| |**10**|30|0|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |**1.5**|**16**|50| \| |25.303|*brakes down to 0.6, bypasses with 0.9*
D2| \| |**10**|30|**2**|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |**1.5**|**16**|50| \| |12.833|*brakes shortly, bypasses with 1*
D3| \| |**10**|30|**4**|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |**1.5**|**16**|50| \| |12.758|*constant speed of 1, drives badly on its lane*
D4| \| |**10**|**60**|**4**|0.1|0.1| \| |0 / 0|2 / 5|2 / 5|1 / 5| \| |0|0|0|0| \| |**1.5**|**16**|50| \| |12.086|*constant speed of 1, drives well in its lane, comes close to obstacle*
D5*| \| |**10**|**60**|**4**|0.1|0.1| \| |0 / 0|2 / 5|**3 / 5.5**|1 / 5| \| |0|0|0|0| \| |**1.5**|**16**|50| \| |13.144|*brakes shortly, bypasses with 1, drives well in lane, keeps distance to obstacle*
D6*| \| |**10**|**60**|**4**|0.1|0.1| \| |0 / 0|2 / 5|**3 / 5.5**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |10.842|*brakes down to 0.8, bypasses with 1, drives well in lane, keeps distance to obstacle*